# 01-3: ReAct


In [ ]:
PROJECT_ID = YOUR_PROJECT_ID  # <--- CHANGE THIS
LOCATION = "us-central1"  # <--- CHANGE THIS
MODEL_NAME = "text-bison" # <--- OPTIONALLY CHANGE HIS. OUTPUTS MAY DIFFER SIGNIFICANTLY

In [ ]:
def call_llm(model, parameters, llm_call, show_activity = True):
  response = model.predict(llm_call, **parameters).text

  if show_activity:
    BOLD = "\033[1m"
    UNFORMAT = "\033[0m\x1B[0m"
    print(f"{BOLD}The call to the LLM:{UNFORMAT}\n{llm_call}\n")
    print(f"{BOLD}The response:{UNFORMAT}")
    print(response)

  return response  # Return to `_` if not needed.

## ReAct Basics

ReAct chains typically have three interleaved parts:
- **Thoughts**: Like in chain of thought, these are waypoints, plans, reasoning, etc. generated by the LLM as it makes progress towards the final output.
- **Actions**: LLM-generated commands, calls, or instructions to access an external system. The external system may be a tool that provides information, but can also be more general (i.e., the action observes or changes the state of an external system).
- **Observations**: A response, feedback, result, etc. from the external system, inserted into an LLM call to generate the next thought.

These three steps are repeated until the LLM completes its task.

Similar to chain-of-thought prompting, this repeated cycle forms an "internal monologue" or "inner speech", but with the important addition of decisions to act and feedback from the actions beyond just the reasoning.

### What a ReAct Chain Looks Like

Before breaking down the LLM calls in a ReAct chain, it helps to see what a complete ReAct chain looks like.

The actions in this chain are Wikipedia lookups, and the observations are snippets from the Wikipedia article.

The original call to the LLM is:
```Question: Who was born first, Ronald Regan or Gerald Ford?```(ignoring instructions, exemplars, etc. for now).

The completed ReAct chain looks like this. Scroll to the right to read the full observations:

```
Question: Who was born first, Ronald Reagan or Gerald Ford?
Thought 1: I need to look up Ronald Reagan and see when he was born.
Action 1: Ronald Reagan<STOP>
Observation 1: Ronald Wilson Reagan (February 6, 1911 – June 5, 2004) was an American politician and actor who served as the 40th president of the United States from 1981 to 1989. A conservative, he was the first president from the West Coast and the first divorced president. Reagan was born in Tampico, Illinois, and raised in Dixon, Illinois. He was educated at Eureka College, where he studied economics and sociology. After graduating, Reagan moved to California, where he became a radio sports announcer. He later moved into acting, appearing in over 50 films. Reagan served as president of the Screen Actors Guild from 1947 to 1952.
Thought 2: Ronald Reagan was born in 1911. I need to look up Gerald Ford and see when he was born.
Action 2: Gerald Ford<STOP>
Observation 2: Gerald Rudolph Ford Jr. ( JERR-əld; born Leslie Lynch King Jr.; July 14, 1913 – December 26, 2006) was an American politician who served as the 38th president of the United States from 1974 to 1977. He previously served as the leader of the Republican Party in the U.S. House of Representatives from 1965 to 1973, when he was appointed the 40th vice president by President Richard Nixon, after Spiro Agnew's resignation. Ford succeeded to the presidency when Nixon resigned in 1974, but was defeated for election to a full term in 1976. Ford is the only person to become U.S. president without winning an election for president or vice president. Ford was born in Omaha, Nebraska and raised in Grand Rapids, Michigan. He attended the University of Michigan, where he played for the school's football team before eventually attending Yale Law School. Afterward, he served in the U.S. Naval Reserve from 1942 to 1946. Ford began his political career in 1949 as the U.S. representative from Michigan's 5
Thought 3: Gerald Ford was born in 1913. 1911 is before 1913. Answer[Ronald Reagan]
```

### Breaking Down a ReAct Chain

The example ReAct chain above is constructed from three LLM calls.

Note the responses in this section have been stripped of extra predicted text, similar to how extra text was stripped in the part 2 tool use discussion.

**Call 1:**
```
Question: Who was born first, Ronald Reagan or Gerald Ford?
Thought 1:
```
**Response 1:**
```
I need to look up Ronald Reagan and see when he was born.
Action 1: Ronald Reagan<STOP>
```

Each LLM call after the first is:
1. The previous LLM call plus
1. The LLM response to the previous call plus
1. The wikipedia lookup result plus
1. "Thought #:"

**Call 2:**

Call 2 is created by concatenating call 1 + response 1 + the result of the wikipedia lookup (in the observation) + "Thought 2:".
```
Question: Who was born first, Ronald Reagan or Gerald Ford?
Thought 1: I need to look up Ronald Reagan and see when he was born.
Action 1: Ronald Reagan<STOP>
Observation 1: Ronald Wilson Reagan (February 6, 1911 – June 5, 2004) was an American politician and actor who served as the 40th president of the United States from 1981 to 1989. A conservative, he was the first president from the West Coast and the first divorced president. Reagan was born in Tampico, Illinois, and raised in Dixon, Illinois. He was educated at Eureka College, where he studied economics and sociology. After graduating, Reagan moved to California, where he became a radio sports announcer. He later moved into acting, appearing in over 50 films. Reagan served as president of the Screen Actors Guild from 1947 to 1952.
Thought 2:
```

**Response 2:**
```
Ronald Reagan was born in 1911. I need to look up Gerald Ford and see when he was born.
Action 2: Gerald Ford<STOP>
```

**Call 3:**

Just like in call 2, we create call 3 by concatenating call 2 + response 2 + the result of the wikipedia lookup + "Thought 3:".

```
Question: Who was born first, Ronald Reagan or Gerald Ford?
Thought 1: I need to look up Ronald Reagan and see when he was born.
Action 1: Ronald Reagan<STOP>
Observation 1: Ronald Wilson Reagan (February 6, 1911 – June 5, 2004) was an American politician and actor who served as the 40th president of the United States from 1981 to 1989. A conservative, he was the first president from the West Coast and the first divorced president. Reagan was born in Tampico, Illinois, and raised in Dixon, Illinois. He was educated at Eureka College, where he studied economics and sociology. After graduating, Reagan moved to California, where he became a radio sports announcer. He later moved into acting, appearing in over 50 films. Reagan served as president of the Screen Actors Guild from 1947 to 1952.
Thought 2: Ronald Reagan was born in 1911. I need to look up Gerald Ford and see when he was born.
Action 2: Gerald Ford<STOP>
Observation 2: Gerald Rudolph Ford Jr. ( JERR-əld; born Leslie Lynch King Jr.; July 14, 1913 – December 26, 2006) was an American politician who served as the 38th president of the United States from 1974 to 1977. He previously served as the leader of the Republican Party in the U.S. House of Representatives from 1965 to 1973, when he was appointed the 40th vice president by President Richard Nixon, after Spiro Agnew's resignation. Ford succeeded to the presidency when Nixon resigned in 1974, but was defeated for election to a full term in 1976. Ford is the only person to become U.S. president without winning an election for president or vice president.
Ford was born in Omaha, Nebraska and raised in Grand Rapids, Michigan. He attended the University of Michigan, where he played for the school's football team before eventually attending Yale Law School. Afterward, he served in the U.S. Naval Reserve from 1942 to 1946. Ford began his political career in 1949 as the U.S. representative from Michigan's 5
Thought 3:
```

Finally, the LLM returns an answer.

**Response 3:**
```
Gerald Ford was born in 1913. 1911 is before 1913. Answer[Ronald Reagan]
```





## Manually Running a ReAct Chain

This section runs a ReAct chain step-by-step.

A few things are required, all in the next code cell:
1. Instructions (context) for the LLM to understand how to do ReAct.
2. At least one exemplar.
3. A tool to execute the LLM's actions.
4. A PaLM API model object to make LLM calls.



In [ ]:
context = """Answer questions with thoughts, actions, and observations.

Think about the next action to take. Then take an action.
All actions are a lookup of Wikipedia.
The Wikipedia action returns the beginning of the best-matching article.
When making a Wikipedia lookup action, end the lookup with <STOP>.
After the Wikipedia action, you will have an observation.
The observation is based on what you learn from the Wikipedia lookup action.
After the observation, begin the loop again with a thought.

Repeat as necessary a thought, taking an action, and having an observation.
Keep repeating as necessary until you know the answer to the question.
When you think you have an answer, return the answer in the format:
"Answer[answer goes here between square brackets]" as part of a thought.
Make sure to capitalize "Answer".

Only use information in the observations to answer the question."""

exemplar = """Example:
Question: Who was born first, Ronald Reagan or Gerald Ford?
Thought 1: I need to look up Ronald Reagan and see when he was born.
Action 1: Ronald Reagan<STOP>
Observation 1: Ronald Wilson Reagan (February 6, 1911 – June 5, 2004) was an American politician and actor who served as the 40th president of the United States from 1981 to 1989. A conservative, he was the first president from the West Coast and the first divorced president. Reagan was born in Tampico, Illinois, and raised in Dixon, Illinois. He was educated at Eureka College, where he studied economics and sociology. After graduating, Reagan moved to California, where he became a radio sports announcer. He later moved into acting, appearing in over 50 films. Reagan served as president of the Screen Actors Guild from 1947 to 1952.
Thought 2: Ronald Reagan was born in 1911. I need to look up Gerald Ford and see when he was born.
Action 2: Gerald Ford<STOP>
Observation 2: Gerald Rudolph Ford Jr. ( JERR-əld; born Leslie Lynch King Jr.; July 14, 1913 – December 26, 2006) was an American politician who served as the 38th president of the United States from 1974 to 1977. He previously served as the leader of the Republican Party in the U.S. House of Representatives from 1965 to 1973, when he was appointed the 40th vice president by President Richard Nixon, after Spiro Agnew's resignation. Ford succeeded to the presidency when Nixon resigned in 1974, but was defeated for election to a full term in 1976. Ford is the only person to become U.S. president without winning an election for president or vice president.
Ford was born in Omaha, Nebraska and raised in Grand Rapids, Michigan. He attended the University of Michigan, where he played for the school's football team before eventually attending Yale Law School. Afterward, he served in the U.S. Naval Reserve from 1942 to 1946. Ford began his political career in 1949 as the U.S. representative from Michigan's 5
Thought 3: Gerald Ford was born in 1913. 1911 is before 1913. Answer[Ronald Reagan]"""

# Code for calling Wikipedia.
import wikipedia
def wiki_tool(query, return_chars = 1000):
  try:
    page = wikipedia.page(query, auto_suggest=False, redirect=True).content
  # If no exact match, take Wikipedia's suggestion.
  except wikipedia.exceptions.PageError as e:
    page = wikipedia.page(query, auto_suggest=True, redirect=True).content
  snippet = page[0:return_chars]
  return snippet

# Initialized PaLM API model.
import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project=PROJECT_ID, location=LOCATION)
# These settings control how deterministic the LLM response is.
parameters = {
    "temperature": 0,
    "max_output_tokens": 256,
    "top_p": 0.8,
    "top_k": 40
}
model = TextGenerationModel.from_pretrained(MODEL_NAME)


The first LLM call is the context, the exemplar, the question, and a label for the first thought.

The action/thought/observation labels at the start of each line are important to ReAct  chains, and increase the likelihood the LLM response sticks to the "script" of interleaved ReAct steps.

In [ ]:
question = "quien nacio antes: Amancio Ortega o Juan Roig" 

llm_call_1 = f"{context}\n\n{exemplar}\n\nQuestion: {question}\nThought 1:"
print(llm_call_1)

In [ ]:
response_1 = call_llm(model, parameters, llm_call_1)

The first and second lines of the response are good. The model generated a reasonable thought and appropriate action.

But just as in the tool use section above, the LLM continues generating garbage text. Remember, LLMs repeatedly predict the next token, and in a ReAct-style LLM call those next tokens are the LLM's prediction of the rest of the ReAct chain.

Just like in the tool use section, extra text is discarded. Only the first two response lines are kept:`Thought 1` and `Action 1`.

In [ ]:
# Only take the first two lines of the response.
# Splitlines returns a list with an item for each line.
response_1 = response_1.splitlines()[0:2]
# Turn response 1 into text from the list so we can concatenate to llm call 1.
response_1 = ("\n").join(response_1)
print(response_1)

Next, query the Wikipedia tool with the LLM's `Action 1` response.

In [ ]:
# Look up the LLM's action in Wikipedia.
wiki_text_1 = wiki_tool("Amancio Ortega")
print(wiki_text_1)

Then construct the next LLM call by adding the Wikipedia tool output as `Observation 1` and then appending `Thought 2:`.

In [ ]:
# Construct the next LLM call.
llm_call_2 = f"{llm_call_1} {response_1}\nObservation 1: {wiki_text_1}\nThought 2:"
print(llm_call_2)

In [ ]:
response_2 = call_llm(model, parameters, llm_call_2)

For the third LLM call in the ReAct chain follow the same procedure as the second call:
1. Take the first two lines of the response.
2. Look up the action in Wikipedia.
3. Assemble the LLM call from the response, the Wikipedia output, and the previous LLM call.

In [ ]:
# Only take the first two lines of the response.
# Splitlines returns a list with an item for each line.
response_2 = response_2.splitlines()[0:2]
# Turn response 1 into text from the list so we can concatenate to llm call 1.
response_2 = ("\n").join(response_2)
print(response_2)

In [ ]:
# Look up the LLM's action in Wikipedia.
wiki_text_2 = wiki_tool("Juan Roig")
print(wiki_text_2)

In [ ]:
# Construct the next LLM call.
llm_call_3 = f"{llm_call_2} {response_2}\nObservation 2: {wiki_text_2}\nThought 3:"
print(llm_call_3)

In [ ]:
response_3 = call_llm(model, parameters, llm_call_3)

And we have an answer!

## A Complete Python Code Snippet for Running ReAct Chains

To use ReAct in an application you need to automate the previous manually-executed steps.

The instructive code snippet below runs ReAct chains. It makes formatted ReAct calls to the LLM, extracts actions, executes actions, detects if the LLM has responded with an answer, and loops.

It's **highly** recommended you walk through the code below and read the comments to better understand how the ReAct chain is automated.

This isn't production-ready code:
1. The snippet is hardcoded to this specific and minimal ReAct example. ReAct chains can look different (more on this later), and useful applications built with ReAct chains  require customized tools.
2. The snippet is brittle, especially the bare-bones Wikipedia tool.
3. The LLM may re-predict previous actions, causing ReAct to infinitely loop. This snippet stops after `max_steps` LLM calls, production ReAct code should catch the loop and attempt to recover.

In [ ]:
import wikipedia

def call_llm(model, parameters, llm_call, show_activity = True):
  # Wraps an LLM call to Vertex, optionally displaying the call and response.
  response = model.predict(llm_call, **parameters).text

  if show_activity:
    BOLD = "\033[1m"
    UNFORMAT = "\033[0m\x1B[0m"
    print(f"{BOLD}The call to the LLM:{UNFORMAT}\n{llm_call}\n")
    print(f"{BOLD}The response:{UNFORMAT}")
    print(response)
  return response  # Return to `_` if not needed.


def wiki_tool(query, return_chars = 1000):
  try:
    page = wikipedia.page(query, auto_suggest=False, redirect=True).content
  # If no exact match, take Wikipedia's suggestion.
  except wikipedia.exceptions.PageError as e:
    page = wikipedia.page(query, auto_suggest=True, redirect=True).content
  snippet = page[0:return_chars]
  return snippet


def wiki_react_chain(model,
                     parameters,
                     context,
                     exemplar,
                     question,
                     max_steps=7,
                     show_activity=False):
  # Call an LLM in a ReACT-style Thought -> Action -> Observation loop.
  # Call the LLM max_steps times or to an answer in the pattern Answer[ans].

  # Construct the first LLM call, teeing up the first thought.
  next_llm_call = f"{context}\n\n{exemplar}\n\nQuestion: {question}\nThought 1:"

  step = 1
  while step <= max_steps:

    if show_activity:
      print(f"\033[1mReAct chain step {step}:\033[0m\x1B[0m")
    llm_response = call_llm(model, parameters, next_llm_call, show_activity)

    # Check for an answer. Look only at the first line of the response, since
    #   the LLM will continue predicting beyond the next thought.
    # This is brittle, it assumes no line breaks in the thought.
    response_first_line = llm_response.splitlines()[0]
    first_line_answer_split = response_first_line.split("Answer[")
    if len(first_line_answer_split) > 1:  # If there's a split on "Answer[".
      # Return the answer, removing the "]" that comes after the answer.
      return first_line_answer_split[1].split("]")[0]

    # If no answer, assume following response line is action.
    response_second_line = llm_response.splitlines()[1]
    """
      Note the hard coded "<STOP>" characters marking the end of the action.
      This isn't strictly necessary if we assume the first line in the LLM
      response is the thought and the second is the action, and that any
      subsequent lines are garbage. But instructing the LLM to explicitly signal
      structure it the response often gives more structurally consistent
      responses, and also makes it easier to detect one way ReAct can fail.
    """
    # Extract the wiki query from the action line of the response.
    wiki_query = response_second_line.split(":")[1].split("<STOP>")[0]
    # Remove leading/trailing whitespace.
    wiki_query = wiki_query.strip()
    if show_activity:
      print(f"\033[1mQuerying wikipedia for: {wiki_query}.\033[0m\x1B[0m")
    wiki_text = wiki_tool(wiki_query)

    # Assemble the next LLM call.
    # Only use the lines of the LLM response with the first thought and action.
    usable_response = f"{response_first_line}\n{response_second_line}"
    # Assemble the wiki response into the observation line.
    obs = f"Observation {step}: {wiki_text}"
    step += 1
    # Previous llm call + the first action and thought in the response +
    # the result of the wikipedia lookup = llm call for next ReAct step.
    # Note that next_llm_call was the last call we made, but we reassign it to
    #   the same variable name so the loop works.
    next_llm_call = f"{next_llm_call} {usable_response}\n{obs}\nThought {step}:"

  # If max_steps exceeded and the loop exits.
  # Would be better to raise an exception.
  return None

An example using the above ReAct chain code snippet.

In [ ]:
import vertexai
from vertexai.language_models import TextGenerationModel

# Outside this notebook, set PROJECT_ID, LOCATION, and MODEL_NAME.
# When running in the notebook, these are set in part 0.
vertexai.init(project=PROJECT_ID, location=LOCATION)
# These settings control how deterministic the LLM response is.
parameters = {
    "temperature": 0,
    "max_output_tokens": 256,
    "top_p": 0.8,
    "top_k": 40
}
model = TextGenerationModel.from_pretrained(MODEL_NAME)

context = """Answer questions with thoughts, actions, and observations.

Think about the next action to take. Then take an action.
All actions are a lookup of wikipedia.
The wikipedia action returns the beginning of the best-matching article.
When making a wikipedia lookup action, end the lookup with <STOP>.
After the wikipedia action, you will make an observation.
The observation is based on what you learn from the wikipedia lookup action.
After the observation, begin the loop again with a thought.

Repeat as necessary a thought, taking an action, and making an observation.
Keep repeating as necessary until you know the answer to the question.
When you think you have an answer, return the answer in the format:
"Answer[answer goes here between square brackets]"
as part of a thought. Make sure to capitalize "Answer".

Only use information in the observations to answer the question."""

exemplar = """Example:
Question: Who was born first, Ronald Reagan or Gerald Ford?
Thought 1: I need to look up Ronald Reagan and see when he was born.
Action 1: Ronald Reagan<STOP>
Observation 1: Ronald Wilson Reagan (February 6, 1911 – June 5, 2004) was an American politician and actor who served as the 40th president of the United States from 1981 to 1989. A conservative, he was the first president from the West Coast and the first divorced president. Reagan was born in Tampico, Illinois, and raised in Dixon, Illinois. He was educated at Eureka College, where he studied economics and sociology. After graduating, Reagan moved to California, where he became a radio sports announcer. He later moved into acting, appearing in over 50 films. Reagan served as president of the Screen Actors Guild from 1947 to 1952.
Thought 2: Ronald Reagan was born in 1911. I need to look up Gerald Ford and see when he was born.
Action 2: Gerald Ford<STOP>
Observation 2: Gerald Rudolph Ford Jr. ( JERR-əld; born Leslie Lynch King Jr.; July 14, 1913 – December 26, 2006) was an American politician who served as the 38th president of the United States from 1974 to 1977. He previously served as the leader of the Republican Party in the U.S. House of Representatives from 1965 to 1973, when he was appointed the 40th vice president by President Richard Nixon, after Spiro Agnew's resignation. Ford succeeded to the presidency when Nixon resigned in 1974, but was defeated for election to a full term in 1976. Ford is the only person to become U.S. president without winning an election for president or vice president.
Ford was born in Omaha, Nebraska and raised in Grand Rapids, Michigan. He attended the University of Michigan, where he played for the school's football team before eventually attending Yale Law School. Afterward, he served in the U.S. Naval Reserve from 1942 to 1946. Ford began his political career in 1949 as the U.S. representative from Michigan's 5
Thought 3: Gerald Ford was born in 1913. 1911 is before 1913. Answer[Ronald Reagan]"""

question = "Quien nació antes, Juan Roig o Amancio Ortega ?"

answer = wiki_react_chain(model,
                          parameters,
                          context,
                          exemplar,
                          question,
                          show_activity = True)
print(answer)


Experiment with changing the `question` above. You may not get great results. This might be due to the brittle Wikipedia tool, but you may see errors with ReAct as well.

Think about how you could improve the performance of the ReAct failures by changing the context or exemplars.

## More ReAct Use Cases

The ReAct pattern does more than answer questions.

With a different context and exemplar, the ReAct code snippet above is adapted for fact checking.

In [ ]:
question = "El numero de tiendas de Inditex es mayor que el de Primark."

context = """You are verifying claims as true or false.
Verify the claim with thoughts, actions, and observations.
Determine if there is an observation that SUPPORTS or REFUTES the claim.

Think about the next action to take to verify the claim. Then take an action.
All actions are a lookup of wikipedia.
The wikipedia action returns the beginning of the best-matching article.
When making a wikipedia lookup action, end the lookup with <STOP>.
After the wikipedia action, you will make an observation.
The observation is based on what you learn from the wikipedia lookup action.
After the observation, begin the loop again with a thought.

Repeat as necessary a thought, taking an action, and making an observation.
Keep repeating as necessary until you reach a conclusion about the claim.
If an observation refutes the claim, return the answer as "Answer[REFUTES]".
If an observation supports the claim, return the answer as "Answer[SUPPORTS]".

Only use information in the observations to answer the question."""

exemplar = """Example:
Claim: Ronald Reagan was born before Gerald Ford.
Thought 1: I need to look up Ronald Reagan and see when he was born.
Action 1: Ronald Reagan<STOP>
Observation 1: Ronald Wilson Reagan (February 6, 1911 – June 5, 2004) was an American politician and actor who served as the 40th president of the United States from 1981 to 1989. A conservative, he was the first president from the West Coast and the first divorced president. Reagan was born in Tampico, Illinois, and raised in Dixon, Illinois. He was educated at Eureka College, where he studied economics and sociology. After graduating, Reagan moved to California, where he became a radio sports announcer. He later moved into acting, appearing in over 50 films. Reagan served as president of the Screen Actors Guild from 1947 to 1952.
Thought 2: Ronald Reagan was born in 1911. I need to look up Gerald Ford and see when he was born.
Action 2: Gerald Ford<STOP>
Observation 2: Gerald Rudolph Ford Jr. ( JERR-əld; born Leslie Lynch King Jr.; July 14, 1913 – December 26, 2006) was an American politician who served as the 38th president of the United States from 1974 to 1977. He previously served as the leader of the Republican Party in the U.S. House of Representatives from 1965 to 1973, when he was appointed the 40th vice president by President Richard Nixon, after Spiro Agnew's resignation. Ford succeeded to the presidency when Nixon resigned in 1974, but was defeated for election to a full term in 1976. Ford is the only person to become U.S. president without winning an election for president or vice president.
Ford was born in Omaha, Nebraska and raised in Grand Rapids, Michigan. He attended the University of Michigan, where he played for the school's football team before eventually attending Yale Law School. Afterward, he served in the U.S. Naval Reserve from 1942 to 1946. Ford began his political career in 1949 as the U.S. representative from Michigan's 5
Thought 3: Gerald Ford was born in 1913. Ronald Reagan was born in 1911. 1911 is before 1913. Ronald Reagan was born before Gerald Ford. Answer[SUPPORTS]"""

answer = wiki_react_chain(model,
                          parameters,
                          context,
                          exemplar,
                          question,
                          show_activity = True)
print(answer)
